# Detect Pine Needle-Leaves in Thermal images
Authors: Jonathan D. Müller & Tamir Dingjan

Loads thermal data from previously extracted raw data, detects reference plates and pine needle-leaves and creates diagnostics graphs

In [ ]:
import numpy as np
from scipy import signal
from tifffile import imread
import glob
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
# Initialization parameters

# Data input
project_path = './'

# Input path
input_path = project_path + '01_rawdata/'

# Diagnostics
diagnostics_path = project_path + '02_diagnostics/'

# Output
output_path = project_path + '03_processed_data/'


# Main variables
debug = True
n     = 1 # Diagnostics graphs every n-th image

# Region to search for needles
needle_xmin = 80
needle_xmax = 320
needle_ymin = 165
needle_ymax = 240

# Region to search for reference plates
plates_xmin = 70
plates_xmax = 190 #150
plates_ymin = 90
plates_ymax = 130

## Functions

In [ ]:
def plot_image_with_peaks(output_path, data, minpeaks, maxpeaks):
    """
    Plot the image with peaks indicated.

    Parameters
    ----------
    output_path : str
        The full path location to save the graph file, including PNG extension.
    data : np.array
        Image data loaded by tifffile.imread.
    minpeaks : dict
        Peak locations generated by analyse_image().
    maxpeaks : dict
        Peak locations generated by analyse_image().

    Returns
    -------
    None.

    """

    plt.figure(figsize=(6,4))
    plt.imshow(data, cmap=plt.cm.gray)
    
    # Plot peaks on image
    plt.scatter(x=minpeaks["x"], y=minpeaks["y"], marker=".", s=0.1, c='r')
    #plt.scatter(x=maxpeaks["x"], y=maxpeaks["y"], marker=".", s=0.1, c='b')
    
    plt.savefig(output_path,
                format="png",
                dpi=300,
                bbox_inches="tight")
    plt.close()
    
def plot_signal(output_path, data, minpeaks, maxpeaks, row_range, col_range, min_col_range, max_col_range):
    """
    Plot the image's signal

    Parameters
    ----------
    output_path : str
        The full path location to save the graph file, including PNG extension.
    data : np.array
        Image data loaded by tifffile.imread.
    minpeaks: np.array
        Selected data minpeaks

    Returns
    -------
    None.

    """
    
    #print(output_path[-31:])
    
    # some constant distance from the median to reject
    dist_from_median = 1.5
    
    # calculate the relevant row
    nrow = int( np.max(row_range) - (np.max(row_range) - np.min(row_range))/4 )
    mask = data[nrow][col_range]
    
    plt.figure(figsize=(6,4))
    plt.plot(np.arange(0,len(mask)), mask)    
    
    if(len(min_col_range) != 0):
        # Minima (infragold)
        min_plate_mask = mask[min_col_range]
        min_peak_filter = np.abs(np.diff(min_plate_mask)) < 0.1
        min_peak_filter_idx = np.where(min_peak_filter)[0] + min_col_range[0]
        min_needle_temps = min_plate_mask[:-1][min_peak_filter]
        # Plot minima (infragold)
        plt.plot(min_peak_filter_idx, mask[min_peak_filter_idx], "x", color='r')
        plt.hlines(np.median(min_needle_temps), 0, len(mask), colors="r", label="IG median", linestyles='dashed')
        plt.hlines(np.mean(min_needle_temps), 0, len(mask), colors="r", label="IG mean")
    
    if(len(max_col_range) != 0):
        # Maxima (acktar)
        max_plate_mask = mask[max_col_range]
        max_peak_filter = np.abs(np.diff(max_plate_mask)) < 0.1
        max_peak_filter_idx = np.where(max_peak_filter)[0] + max_col_range[0]
        max_needle_temps = max_plate_mask[:-1][max_peak_filter]
        # Plot maxima (acktar)
        plt.plot(max_peak_filter_idx, mask[max_peak_filter_idx], "x", color='b')
        plt.hlines(np.median(max_needle_temps), 0, len(mask), colors="b", label="AC median", linestyles='dashed')
        plt.hlines(np.mean(max_needle_temps), 0, len(mask), colors="b", label="AC mean")
    
    #temps = np.mean(min_needle_temps)      
    plt.legend(loc="best")   
    plt.savefig(output_path,
                format="png",
                dpi=300,
                bbox_inches="tight")
    plt.close()
    
    
    
def get_peaks(mask, median_range, amp_factor):
    """
    Gets the peak locations and corresponding temperature values for the 
    provided 1D slice of data. Only peaks with temperature values within
    the provided range of the median peak temperature value are reported.
    In early testing, this was shown to be helpful in removing peaks
    detected within the background noise.

    Parameters
    ----------
    mask : np.array
        1D array of temperature data points.
    median_range : float
        Only return peaks with temperatures within this tolerance of the median
        temperature value (calculated from all peaks). 
    amp_factor : float
        Use this to determine the minimum peak prominence relative to the 
        total signal amplitude.

    Returns
    -------
    peaks : np.array
        Locations of detected peaks.
    temps : np.array
        Peak temperature values.

    """
    
    # select prominence minimum as 1/3 of total signal amplitude
    amplitude = (mask.max() - mask.min())/amp_factor
    # Save minimum and maximum peaks
    peaks, _ = signal.find_peaks(mask, width=(1,5), prominence=amplitude)
           
    # Return the peak locations and the peak temperatures
    if (len(peaks) != 0):
        peak_filter = np.abs(mask[peaks]-np.median(mask[peaks])) < median_range
        peaks_out = peaks[peak_filter]
        temps = mask[peaks[peak_filter]]
    else:
        peaks_out = []
        temps = np.nan
    
    return peaks_out, np.asarray(temps)


def analyse_ref_plates(path, row_range, col_range, output_path, debug=False):
    """
    Measure the minimum and maximum temperatures of reference places.

    Parameters
    ----------
    path : str
        Full path to the data file to be analysed.
    row_range : iterable
        Row indices of the data to analyse.
    col_range : iterable
        Column indices of the data to analyse.
    output_path : str
        Full path to the graph file to be plotted.

    Returns
    -------
    min_temps : np.array
        Minimum temperatures within the reference plate range.
    max_temps : np.array
        Maximum temperatures within the reference plate range.
        
    """
    
    # Load the data
    data = imread(path)
    
    min_temps = []
    min_peaks = {}
    min_peaks["x"] = []
    min_peaks["y"] = []
    
    max_temps = []
    max_peaks = {}
    max_peaks["x"] = []
    max_peaks["y"] = []
    
    prev_junction_loc = 0
    
    for row in row_range:
        # Select the region - a single row along the entire col range
        mask = data[row][col_range]
        
         # Define cold plate as the minimum value of the signal
        min_loc = np.where(mask==mask.min())[0][0]
        
        # Find the plate junction in the previous 60 pixels
        section = np.diff(mask[:min_loc])
        if(len(section) == 0): # No plate junction could be found, next row
            #print("No plate junction in "+path[-24:]+", row: "+str(row))
            continue
        else:
            plate_junction_loc = np.where(section == section.min())[0][0]
            if(row != (row_range[0])): # The first row is the index of approximately where the plate junction is to be looked for
                if((plate_junction_loc < (prev_junction_loc-10)) or (plate_junction_loc > (prev_junction_loc+10))): # the junction is outside the realistic range
                    continue # Skip to next row
            else: # If it was the first row, save the location of the junction for the next run
                prev_junction_loc = plate_junction_loc
                
        
        # Select ranges for temperature evaluation
        junction_buffer = 10
        plate_width = 20
        num_points = 4
        
        # Find the Acktar plate
        ac_plate_start = plate_junction_loc - junction_buffer - plate_width
        ac_plate_end = plate_junction_loc - junction_buffer
        # Old: Maximum values on the plate. Problem: this adds points that are needles in front of the plate.
        #ac_samples = np.argsort(mask[ac_plate_start:ac_plate_end])[-num_points:] # indices of maximal plate values
        #ac_plate_temps = mask[ac_plate_start:ac_plate_end][ac_samples]
        #max_temps.append(ac_plate_temps)
        
        # New: Pick the highest values of what is the plate (nothing else)
        #peak_filter = np.abs(np.diff(mask[ac_plate_start:ac_plate_end])) < 0.08 # removes outliers like needles
        #ac_plate_temps = mask[ac_plate_start:(ac_plate_end-1)][peak_filter] # all values of plate
        #ac_samples = np.argsort(ac_plate_temps)[-num_points:] # get indices of highest values
        #ac_plate_temps = ac_plate_temps[ac_samples] # get values of highest values
        #max_temps.append(ac_plate_temps)
        
        # New: Pick random values of what is the plate (nothing else)
        peak_filter = np.abs(np.diff(mask[ac_plate_start:ac_plate_end])) < 0.06 # removes outliers like needles (0.08)
        ac_plate_temps = mask[ac_plate_start:(ac_plate_end-1)][peak_filter] # all values of plate
        if(len(np.where(peak_filter)[0]) > num_points): # With few samples, I have to remove the entire sampling line
            ac_samples = np.random.choice(np.where(peak_filter)[0], size=num_points, replace=False) # Randomly pick 3 index values from the plate values
        else:
            #print("Not enough Acktar samples in "+path[-24:]+", row: "+str(row))
            ac_samples = []
        ac_plate_temps = mask[ac_plate_start:(ac_plate_end-1)][ac_samples] # get temperature of random points
        max_temps.append(ac_plate_temps)
        
        # Find the Infragold plate
        ig_plate_start = plate_junction_loc + junction_buffer
        ig_plate_end = plate_junction_loc + junction_buffer + plate_width
        ig_samples = np.argsort(mask[ig_plate_start:ig_plate_end])[:num_points] # indices of minimal plate values
        ig_plate_temps = mask[ig_plate_start:ig_plate_end][ig_samples]
        min_temps.append(ig_plate_temps)
        
        # Store locations of sampling points
        for i in ac_samples:
            max_peaks["x"].append(i+ac_plate_start+col_range[0])
            max_peaks["y"].append(row)
              
        for i in ig_samples:
            min_peaks["x"].append(i+ig_plate_start+col_range[0])
            min_peaks["y"].append(row)
        

    # Plot the peaks onto the source image
    if(debug):
        plot_image_with_peaks(output_path, data, min_peaks, max_peaks)
        # Plot the signal
        ig_col_range = np.arange(ig_plate_start, ig_plate_end, 1)
        ac_col_range = np.arange(ac_plate_start, ac_plate_end, 1)
        if(len(ig_col_range) != 0 or len(ac_col_range) != 0): # Do not plot signal if plate_col_range is empty
            plot_signal(output_path[:-10]+"_signal.png", data, min_peaks, max_peaks, row_range, col_range, ig_col_range, ac_col_range)
        
        #if(len(ac_col_range) != 0): # Do not plot signal if plate_col_range is empty
        #    plot_signal(output_path[:-10]+"_signal_ac.png", data, max_peaks, row_range, col_range, ac_col_range)

    # Return the temperature data and peak counts
    return (min_temps, max_temps)



def analyse_needles(path, row_range, col_range, median_range, amp_factor, output_path, debug=False):
    """
    Runs the peak detection on multiple rows of the provided data file,
    scanning for both maximum and minimum peaks. 

    Also invokes plot_image_with_peaks() to generate a figure of the input
    data with peaks shown.

    Parameters
    ----------
    path : str
        Full path to the data file to be analysed.
    row_range : iterable
        Row indices of the data to analyse.
    col_range : iterable
        Column indices of the data to analyse.
    median_range : float
        Tolerance around the median temperature for peak filtering.
    amp_factor : float
        Ration of signal amplitude to peak prominence for peak filtering.
    output_path : str
        Full path to the graph file to be plotted.

    Returns
    -------
    tuple
        A tuple containing the following:
        min_temps : np.array
            Temperatures from minimum peaks
        len(min_peaks["x"]) : int
            Number of minimum peaks detected
        max_temps : np.array
            Temperatures from maximum peaks
        len(max_peaks["x"])) : int
            Number of maximum peaks detected

    """
    # Load the data
    data = imread(path)
    
    min_temps = []
    min_peaks = {}
    min_peaks["x"] = []
    min_peaks["y"] = []
    
    max_temps = []
    max_peaks = {}
    max_peaks["x"] = []
    max_peaks["y"] = []
    
    for row in row_range:
        
        # Select the region - a single row along the entire col range
        mask = data[row][col_range]
        
        # Get the min peaks - invert mask to select minima
        min_peaks_row, min_temps_row = get_peaks(-mask, median_range, amp_factor)
        
        # If we have peaks and temp data, store them
        if (len(min_peaks_row) != 0):
            min_temps.append(min_temps_row)
            # Store the peak locations
            for i in min_peaks_row:
                min_peaks["x"].append(i+col_range[0])
                min_peaks["y"].append(row)
        # If there were no peaks, store the mean temperature of this row
        else:
            min_temps.append(np.asarray(np.mean(mask)))
        
        # Get the max peaks
        max_peaks_row, max_temps_row = get_peaks(mask, median_range, amp_factor)
        
        # If we have peaks and temp data, store them
        if (len(max_peaks_row) != 0):
            max_temps.append(max_temps_row)
            # Store the peak locations
            for i in max_peaks_row:
                max_peaks["x"].append(i+col_range[0])
                max_peaks["y"].append(row)
        # If there were no peaks, store the mean temperature of this row
        else:
            max_temps.append(np.mean(mask))
        
    # Plot the peaks onto the source image
    if(debug):
        plot_image_with_peaks(output_path, data, min_peaks, max_peaks)
    
    # Return the temperature data and peak counts
    return (min_temps, len(min_peaks["x"]), max_temps, len(max_peaks["x"]))

## Processing pipeline

In [ ]:
# Data list
files = glob.glob(input_path + "**/*.tif", recursive=True)

# Choose which region to analyse
#-------------------------------
# Region to search for needles
needle_row_range = np.arange(needle_ymin, needle_ymax, 1)
needle_col_range = np.arange(needle_xmin, needle_xmax, 1)
# Select peaks within 1.5 degrees of median peak temp
median_range = 1.5
# Factor of signal amplitude to use for minimum peak prominence
amp_factor = 3

# Setup data
needle_mins = []
needle_mins_median = []
needle_min_sd = []
needle_min_peak_count = []
needle_maxs = []
needle_maxs_median = []
needle_max_sd = []
needle_max_peak_count = []

# Region to search for reference plates
plate_row_range = np.arange(plates_ymin, plates_ymax, 1)
plate_col_range = np.arange(plates_xmin, plates_xmax, 1) 

# Setup data
plate_ig = []
plate_ig_sd = []
plate_ac = []
plate_ac_sd = []

for x_i, x in enumerate(files):
    filename = os.path.basename(x)[5:20]
    # Diagnostics every 100th image
    if(x_i % n == 0):
        print('{:.4}'.format(x_i*100/len(files)) + "% \t" + filename)
        
    needle_graph_name = os.path.splitext(os.path.basename(x))[0]+"_needle.png"
    if(x_i % 100 == 0): # Create a diagnostic graph output every 100th image
        output = analyse_needles(x, needle_row_range, needle_col_range, median_range, amp_factor, diagnostics_path+needle_graph_name, debug=True)
    else:
        output = analyse_needles(x, needle_row_range, needle_col_range, median_range, amp_factor, diagnostics_path+needle_graph_name, debug=debug)
    needle_mins.append(np.nanmean(np.abs(np.concatenate([x.flatten() for x in output[0]]))))
    needle_mins_median.append(np.nanmedian(np.abs(np.concatenate([x.flatten() for x in output[0]]))))
    needle_min_sd.append(np.nanstd(np.abs(np.concatenate([x.flatten() for x in output[0]]))))
    needle_min_peak_count.append(output[1])
    needle_maxs.append(np.nanmean(np.abs(np.concatenate([x.flatten() for x in output[2]]))))
    needle_maxs_median.append(np.nanmedian(np.abs(np.concatenate([x.flatten() for x in output[2]]))))
    needle_max_sd.append(np.nanstd(np.abs(np.concatenate([x.flatten() for x in output[2]]))))
    needle_max_peak_count.append(output[3])
    
    plate_graph_name = os.path.splitext(os.path.basename(x))[0]+"_plate.png"
    if(x_i % n == 0): # Create a diagnostic graph output every 100th image
        output = analyse_ref_plates(x, plate_row_range, plate_col_range, diagnostics_path+plate_graph_name, debug=True)
    else:
        output = analyse_ref_plates(x, plate_row_range, plate_col_range, diagnostics_path+plate_graph_name, debug=debug)
    #print(len([x.flatten() for x in output[0]]))
    if(len([x.flatten() for x in output[0]]) > 5):
        plate_ig.append(np.median(np.concatenate([x.flatten() for x in output[0]])))
        #plate_ig.append(np.mean(np.concatenate([x.flatten() for x in output[0]])))
        plate_ig_sd.append(np.std(np.concatenate([x.flatten() for x in output[0]])))
        plate_ac.append(np.median(np.concatenate([x.flatten() for x in output[1]])))
        #plate_ac.append(np.mean(np.concatenate([x.flatten() for x in output[1]])))
        plate_ac_sd.append(np.std(np.concatenate([x.flatten() for x in output[1]])))
    else:
        print("Less than 5 arrays in: " + filename)
        plate_ig.append(np.nan)
        plate_ig_sd.append(np.nan)
        plate_ac.append(np.nan)
        plate_ac_sd.append(np.nan)
    
    
frame = {"datetime":[os.path.basename(x)[5:20] for x in files],
         "needle_min":needle_mins,
         "needle_min_median":needle_mins_median, 
         "needle_min.sd":needle_min_sd, 
         "needle_min.n":needle_min_peak_count,
         "needle_max":needle_maxs,
         "needle_max_median":needle_maxs_median, 
         "needle_max.sd":needle_max_sd,
         "needle_max.n":needle_max_peak_count,
         "ig":plate_ig, 
         "ig.sd":plate_ig_sd, 
         "ac":plate_ac, 
         "ac.sd":plate_ac_sd}

df = pd.DataFrame(frame)
df.sort_values('datetime', inplace = True)
df.drop_duplicates(inplace = True) 
df.to_csv(output_path+"ircam.csv", index=False)

print("Done...")